### NLP building a Chat bot

We as a group descided to build a chat bot that gives information about the resturants presenets on the premises of the university of surrey 

We are considering the following restuarnats
    
    Lakeside
    Starbucks
    Teas and Tees
    Theatre   
    Vet school cafe
    Pizzaman
    Youngs
    Hillside
    Wates House

The main target is to provide information on timings , location and type of the resturant. Our intitial approach is to use intent and NER to understand the request and build simple dialogue flow using those 

We developed a data set for training the Intent classifier and also built a custom NER model using a blank spacy model


Model serving options info here

In [ ]:
def train_intent(data,train_flag):
    #Importing the libraries for intent training
    import wget
    import numpy as np
    import json
    import numpy as np
    import os
    import pickle
    from sklearn.model_selection import train_test_split
    from keras.preprocessing.text import Tokenizer
    from keras.preprocessing.sequence import pad_sequences
    from sklearn.preprocessing import OneHotEncoder,LabelEncoder
    #if loop to verify whether the ".txt" file is present or not, to satisfy the CI/CD pipeline
    if (not (os.path.exists('./glove.6B.100d.txt'))):
        print("inesle ")
        url= 'https://www.dropbox.com/s/a247ju2qsczh0be/glove.6B.100d.txt?dl=1'
        wget.download(url)
    #getting the json file keys from the data 
    keyss = list(data.keys())
    array11=np.array(data[keyss[0]])
    array12=np.array(data[keyss[1]])
    array13=np.array(data[keyss[2]])
    array14=np.array(data[keyss[3]])
    array15=np.array(data[keyss[4]])
    #Merging all the keys into one variable for training and testing
    val=np.concatenate([array11, array12,array13,array14,array15])
    text=[]
    labels=[]
    for i in  range(len(val)):
        text.append(val[i][0])
        labels.append(val[i][1])
    train_txt,test_txt,train_label,test_labels = train_test_split(text,labels,test_size = 0.2)
    #Implementing bag of words to collect all the words
    ls=[]
    for c in train_txt:
        ls.append(len(c.split()))
    maxLen=int(np.percentile(ls, 98))
    embeddings_index={}
    with open('glove.6B.100d.txt', encoding='utf-8') as f:
        for line in f:
            values=line.split()
            word=values[0]
            coefs=np.asarray(values[1:], dtype='float32')
            embeddings_index[word]=coefs
    all_embs=np.stack(embeddings_index.values())
    emb_mean, emb_std= all_embs.mean(), all_embs.std()
    max_num_words=40000
    embedding_dim=len(embeddings_index['the'])
    classes=np.unique(labels)
    tokenizer=Tokenizer(num_words=max_num_words)
    tokenizer.fit_on_texts(train_txt)
    train_sequences=tokenizer.texts_to_sequences(train_txt)
    train_sequences=pad_sequences(train_sequences, maxlen=maxLen, padding='post')
    test_sequences=tokenizer.texts_to_sequences(test_txt)
    test_sequences=pad_sequences(test_sequences, maxlen=maxLen, padding='post')
    word_index=tokenizer.word_index
    num_words=min(max_num_words, len(word_index))+1
    embedding_matrix=np.random.normal(emb_mean, emb_std, (num_words, embedding_dim))
    for word, i in word_index.items():
        if i >= max_num_words:
            break
        embedding_vector=embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i]=embedding_vector
    #Initialising One Hot Encoding
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(classes)
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoder.fit(integer_encoded)
    train_label_encoded = label_encoder.transform(train_label)
    train_label_encoded = train_label_encoded.reshape(len(train_label_encoded), 1)
    train_label = onehot_encoder.transform(train_label_encoded)
    test_labels_encoded = label_encoder.transform(test_labels)
    test_labels_encoded = test_labels_encoded.reshape(len(test_labels_encoded), 1)
    test_labels = onehot_encoder.transform(test_labels_encoded)
    #Dumping the pickle file
    with open('utils/tokenizer.pkl', 'wb') as file:
        pickle.dump(tokenizer, file)
    with open('utils/label_encoder.pkl', 'wb') as file:
        pickle.dump(label_encoder, file)
    with open('utils/classes.pkl', 'wb') as file:
        pickle.dump(classes, file)
    if train_flag == 1:
        intent_train_model(num_words,train_sequences,train_label,test_sequences,test_labels,embedding_matrix,classes)
    return("Intent training done")
    

In [ ]:
def intent_train_model(num_words,train_sequences,train_label,test_sequences,test_labels,embedding_matrix,classes):
    #print("in training")
    #Importing libraries for feed forward neural network
    from keras.models import Sequential
    from keras.layers import Dense, Input, Dropout, LSTM, Activation, Bidirectional,Embedding
    import pickle
    import json
    model = Sequential()
    model.add(Embedding(num_words, 100, trainable=False,input_length=train_sequences.shape[1], weights=[embedding_matrix]))
    model.add(Bidirectional(LSTM(256, return_sequences=True, recurrent_dropout=0.1, dropout=0.1), 'concat'))
    model.add(Dropout(0.3))
    model.add(LSTM(256, return_sequences=False, recurrent_dropout=0.1, dropout=0.1))
    model.add(Dropout(0.3))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(classes.shape[0], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    print(model.summary())
    history = model.fit(train_sequences, train_label, epochs = 15,
              batch_size = 20, shuffle=True,
              validation_data=[test_sequences, test_labels])
    model.save('models/intents.h5')
    return("Intent training done")
    
    

In [43]:
def train_ner(f): 
    import spacy
    from spacy.tokens import DocBin
    from tqdm import tqdm
    import json
    import os 
    if (os.path.exists('./model-best')):
        nlp = spacy.load("model-best") # if already a trained model exists build upon it 
    else :
        nlp = spacy.blank("en") # if not load a blank spacy model
    db = DocBin()

    TRAIN_DATA = json.load(f)

    for text, annot in tqdm(TRAIN_DATA['annotations']): 
        doc = nlp.make_doc(text) 
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents 
        db.add(doc)

    db.to_disk("./training_data.spacy") # save the docbin object

    ! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

    ! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy
    
    return("Training done and New ner is ready")

This following can be used for CI CD as well we can run and retrain models once we collect enough data 

In [ ]:
#training all the models and save them 

import json
with open('CW11.json') as file:
    data=json.loads(file.read())
train_flag=1
print(train_intent(data,train_flag))#training intent file 
f = open('version3.json')
print(train_ner(f)) # traingin ner 

In [2]:
class ner_function():
    def load_ner():
        import spacy
        from spacy.tokens import DocBin
        from tqdm import tqdm
        global nlp_ner
        nlp_ner = spacy.load("model-best")
    def get_ner(sent,tag):
        flag =0
        doc = nlp_ner(sent) 
        for ent in doc.ents:
            if ent.label_ == tag:
                return (ent.text)
                flag =1
        if flag == 0:
            return("NO_ENTITY_FOUND")
    

In [3]:
class intent_function():
    from keras.preprocessing.sequence import pad_sequences
    def load_intent():
        import pickle
        import numpy as np
        from keras.models import load_model
        from keras.preprocessing.sequence import pad_sequences
        global model
        model=load_model('models/intents.h5')
        with open('utils/classes.pkl', 'rb') as file:
            global classes
            classes=pickle.load(file)
        with open('utils/tokenizer.pkl', 'rb') as file:
            global tokenizer
            tokenizer=pickle.load(file)
        with open('utils/label_encoder.pkl', 'rb') as file:
            global label_encoder
            label_encoder=pickle.load(file)
    def get_intent(senttt):
        from keras.preprocessing.sequence import pad_sequences
        import numpy as np
        class IntentClassifier:
            from keras.preprocessing.sequence import pad_sequences
            def __init__(self, classes, model, tokenizer, label_encoder):
                self.classes=classes
                self.classifier=model
                self.tokenizer=tokenizer
                self.label_encoder=label_encoder
            def predict_intent(self, text):
                from keras.preprocessing.sequence import pad_sequences
                self.text=[text]
                self.test_keras=self.tokenizer.texts_to_sequences(self.text)
                self.test_keras_sequence=pad_sequences(self.test_keras, maxlen=8, padding='post')
                self.pred=self.classifier.predict(self.test_keras_sequence)
                return label_encoder.inverse_transform(np.argmax(self.pred, 1))[0]
        nlu=IntentClassifier(classes, model, tokenizer, label_encoder)
        return(nlu.predict_intent(senttt))
    

In [4]:
def is_question(senttt):
    from nltk.tokenize import word_tokenize
    question_words = ["what", "why", "when", "where", 
             "name", "is", "how", "do", "does", 
             "which", "are", "could", "would", 
             "should", "has", "have", "whom", "whose", "don't"]

    question = senttt
    question = question.lower()
    question = word_tokenize(question)

    if any(x in question[0] for x in question_words):
        return (1)
    else:
        return (0)

In [6]:
def get_info(rest_name,info_needed):
    import ast
    file = open("dictionary.txt", "r")
    contents = file.read()
    dictionary = ast.literal_eval(contents)
    information = dictionary[rest_name][info_needed]
    return information

In [5]:
def get_response(intent_detected,sentence):
    import random 
    resp_greet =['Hello', 'Heya','Hi','Hi! How are you doing?','Hey!how are you doing?','Hola','Hello! hope you are doing well','Hello! Whats up?','Hie! how have you been?','Hie!']
    resp_goobye =['Bye Bye!', 'Goodbye','See you!','Bye, it was nice talking to you','Goodbye!, have a nice day','I hope this works','thankyou for having a chat','thank you! I hope this chat helped you']
    resp_timings =['time','timeess']
    resp_type=['typeeee','typessss']
    resp_locc =['locccc','location']
    sorry = "Sorry i dont understand the question i will prepare well next time"
    if intent_detected == 'greeting':
        return (random.choice(resp_greet))
    elif intent_detected == 'goodbye':
        return (random.choice(resp_goobye))
    elif intent_detected == 'info_timings':
        namess = ner_function.get_ner(sentence,'RESTAURANT_NAME')
        if namess == "NO_ENTITY_FOUND" :
            return (sorry)
        elif 'open' in sentence:
            ress = namess + ' opening timings are ' + get_info(namess,'info_open')
            return (ress)
        elif 'close' in sentence:
            ress = namess + ' Closing timings are ' + get_info(namess,'info_close')
            return (ress)
    elif intent_detected == 'info_type':
        namess = ner_function.get_ner(sentence,'RESTAURANT_NAME')
        if namess == "NO_ENTITY_FOUND":
            return (sorry)
        else :
            ress = namess + ' is  ' + get_info(namess,'type')
            return (ress)     
    elif intent_detected == 'info_location':
        namess = ner_function.get_ner(sentence,'RESTAURANT_NAME')
        if namess == "NO_ENTITY_FOUND":
            return (sorry)
        else :          
            ress = namess + ' is located  ' + get_info(namess,'location')
            return (ress)   
    

In [7]:
def store_logs(sent_list,resp_list,intent_list):
    import pandas as pd
    import os
    df = pd.DataFrame(list(zip(sent_list, resp_list,intent_list)),columns =['input', 'response','intent'])
    if (not (os.path.exists('./logs.csv'))):
        df.to_csv("logs.csv")
    else :
        df1 = pd.read_csv('logs.csv')
        df = pd.concat([df, df1])
        df.to_csv("logs.csv")
    

In [8]:
bot_name ='infomania'
intent_function.load_intent()
ner_function.load_ner()
print("Hi i am INFORMIA and i will help you find information about restuarants in Univertsity of surrey")
print("Disclaimer :: ---- I am still learning and will get better over time please dont be MAD AT ME") 
print("Let's chat! (type 'quit' to exit)")
bot_name ='infomania'
sent_list=[]
resp_list=[]
intent_list =[]
curr_log=[]
while True:
    # sentence = "do you use credit cards?"
    sentence = input("You: ")
    curr_log.append(('You :' + sentence))
    sent_list.append(sentence)
    if sentence == "quit":
        store_logs(sent_list,resp_list,intent_list)
        break
    
    else:
        intent_detected = intent_function.get_intent(sentence)
        intent_list.append(intent_detected)
        response = get_response(intent_detected,sentence)
        resp_list.append(response)
        print(f"{bot_name}:",response )
        curr_log.append(('infomania :' + response))

Hi i am INFORMIA and i will help you find information about restuarants in Univertsity of surrey
Disclaimer :: ---- I am still learning and will get better over time please dont be MAD AT ME
Let's chat! (type 'quit' to exit)


You:  hello


infomania: Hola


You:  where is hillside


infomania: hillside is located  stag hill


You:  why ?


infomania: Heya


You:  good byr


infomania: Hi


You:  quit


In [10]:
print(curr_log)

['You :hello', 'infomania :Hola', 'You :where is hillside', 'infomania :hillside is located  stag hill', 'You :why ?', 'infomania :Heya', 'You :good byr', 'infomania :Hi', 'You :quit']
